# Structured and time series data

This notebook contains an implementation of the third place result in the Rossman Kaggle competition as detailed in Guo/Berkhahn's [Entity Embeddings of Categorical Variables](https://arxiv.org/abs/1604.06737).

The motivation behind exploring this architecture is it's relevance to real-world application. Most data used for decision making day-to-day in industry is structured and/or time-series data. Here we explore the end-to-end process of using neural networks with practical structured data problems.

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

PATH='../data/'

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Load datasets

In [3]:
table_names = ['train', 'test']

We'll be using the popular data manipulation framework `pandas`. Among other things, pandas allows you to manipulate tables/data frames in python as one would in a database.

We're going to go ahead and load all of our csv's as dataframes into the list `tables`.

In [4]:
tables = [pd.read_csv(f'{PATH}{fname}.tsv', low_memory=False, sep='\t') for fname in table_names]

In [5]:
from IPython.display import HTML, display

We can use `head()` to get a quick look at the contents of each table:
* train: Contains store information on a daily basis, tracks things like sales, customers, whether that day was a holdiay, etc.
* store: general info about the store including competition, etc.
* store_states: maps store to state it is in
* state_names: Maps state abbreviations to names
* googletrend: trend data for particular week/state
* weather: weather conditions for each state
* test: Same as training table, w/o sales and customers


In [6]:
for t in tables: display(t.head())

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


This is very representative of a typical industry dataset.

The following returns summarized aggregate information to each table accross each field.

In [7]:
for t in tables: display(DataFrameSummary(t).summary())

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
count,1.48254e+06,NaN,1.48254e+06,NaN,NaN,1.48254e+06,1.48254e+06,NaN
mean,741267,NaN,1.90738,NaN,NaN,26.7375,0.447274,NaN
std,427971,NaN,0.903159,NaN,NaN,38.5861,0.497212,NaN
min,0,NaN,1,NaN,NaN,0,0,NaN
25%,370634,NaN,1,NaN,NaN,10,0,NaN
50%,741267,NaN,2,NaN,NaN,17,0,NaN
75%,1.1119e+06,NaN,3,NaN,NaN,29,1,NaN
max,1.48253e+06,NaN,5,NaN,NaN,2009,1,NaN
counts,1482535,1482535,1482535,1476208,849853,1482535,1482535,1482531
uniques,1482535,1225273,5,1287,4809,828,2,1281426


,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
count,693359,NaN,693359,NaN,NaN,693359,NaN
mean,346679,NaN,1.9061,NaN,NaN,0.447719,NaN
std,200156,NaN,0.903378,NaN,NaN,0.49726,NaN
min,0,NaN,1,NaN,NaN,0,NaN
25%,173340,NaN,1,NaN,NaN,0,NaN
50%,346679,NaN,2,NaN,NaN,0,NaN
75%,520018,NaN,3,NaN,NaN,1,NaN
max,693358,NaN,5,NaN,NaN,1,NaN
counts,693359,693359,693359,690301,397834,693359,693359
uniques,693359,601117,5,1223,3900,2,609555


## Data Cleaning / Feature Engineering

As a structured data problem, we necessarily have to go through all the cleaning and feature engineering, even though we're using a neural network.

In [8]:
train, test = tables

In [9]:
len(train),len(test)

(1482535, 693359)

We turn state Holidays to booleans, to make them more convenient for modeling. We can do calculations on pandas fields using notation very similar (often identical) to numpy.

In [10]:
# train.category_name = train.category_name.fillna('missing').astype('category')
# train.brand_name = train.brand_name.fillna('missing').astype('category')
# train.item_condition_id = train.item_condition_id.astype('category')

# test.category_name = test.category_name.fillna('missing').astype('category')
# test.brand_name = test.brand_name.fillna('missing').astype('category')
# test.item_condition_id = test.item_condition_id.astype('category')

It's usually a good idea to back up large tables of extracted / wrangled features before you join them onto another one, that way you can go back to it easily if you need to make changes to it.

In [11]:
# combined = pd.concat([train, test])

In [12]:
train.to_feather(f'{PATH}train_pp1')
test.to_feather(f'{PATH}test_pp1')

We now have our final set of engineered features.

While these steps were explicitly outlined in the paper, these are all fairly typical feature engineering steps for dealing with time series data and are practical in any similar setting.

## Create features

In [73]:
train = pd.read_feather(f'{PATH}train_pp1')
test = pd.read_feather(f'{PATH}test_pp1')

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [74]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 8 columns):
train_id             1482535 non-null int64
name                 1482535 non-null object
item_condition_id    1482535 non-null int64
category_name        1476208 non-null object
brand_name           849853 non-null object
price                1482535 non-null float64
shipping             1482535 non-null int64
item_description     1482531 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 90.5+ MB


In [75]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 7 columns):
test_id              693359 non-null int64
name                 693359 non-null object
item_condition_id    693359 non-null int64
category_name        690301 non-null object
brand_name           397834 non-null object
shipping             693359 non-null int64
item_description     693359 non-null object
dtypes: int64(3), object(4)
memory usage: 37.0+ MB


In [76]:
train.head().T.head(40)

,0,1,2,3,4
train_id,0,1,2,3,4
name,MLB Cincinnati Reds T Shirt Size XL,Razer BlackWidow Chroma Keyboard,AVA-VIV Blouse,Leather Horse Statues,24K GOLD plated rose
item_condition_id,3,3,1,1,1
category_name,Men/Tops/T-shirts,Electronics/Computers & Tablets/Components & P...,Women/Tops & Blouses/Blouse,Home/Home Décor/Home Décor Accents,Women/Jewelry/Necklaces
brand_name,None,Razer,Target,None,None
price,10,52,10,35,44
shipping,1,0,1,1,0
item_description,No description yet,This keyboard is in great condition and works ...,Adorable top with a hint of lace and a key hol...,New with tags. Leather horses. Retail for [rm]...,Complete with certificate of authenticity


Now that we've engineered all our features, we need to convert to input compatible with a neural network.

This includes converting categorical variables into contiguous integers or one-hot encodings, normalizing continuous features to standard normal, etc...

In [77]:
## Other Categories are too often!!

In [78]:
train['x'] = 1
train['x'] = train['x'].map(lambda x: np.random.random(x)[0])

test['x'] = 1
test['x'] = test['x'].map(lambda x: np.random.random(x)[0])

train.price = train.price.astype('float32')

In [79]:
cat_vars = ['item_condition_id','category_name', 'brand_name', 'shipping']

contin_vars = ['x']

n = len(train); n

1482535

In [80]:
train.columns

Index(['train_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'price', 'shipping', 'item_description', 'x'],
      dtype='object')

In [81]:
test.columns

Index(['test_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'shipping', 'item_description', 'x'],
      dtype='object')

In [82]:
dep = 'price'
train = train[cat_vars+contin_vars+[dep]].copy()

In [83]:
train.head()

,item_condition_id,category_name,brand_name,shipping,x,price
0,3,Men/Tops/T-shirts,None,1,0.897656,10.0
1,3,Electronics/Computers & Tablets/Components & P...,Razer,0,0.728970,52.0
2,1,Women/Tops & Blouses/Blouse,Target,1,0.425310,10.0
3,1,Home/Home Décor/Home Décor Accents,None,1,0.105122,35.0
4,1,Women/Jewelry/Necklaces,None,0,0.099759,44.0


In [84]:
test.columns

Index(['test_id', 'name', 'item_condition_id', 'category_name', 'brand_name',
       'shipping', 'item_description', 'x'],
      dtype='object')

In [85]:
test[dep] = 0
test = test[cat_vars+contin_vars+[dep, 'test_id']].copy()
test.rename(columns={'test_id': 'Id'}, inplace=True)

In [86]:
for v in cat_vars: train[v] = train[v].astype('category').cat.as_ordered()

In [87]:
apply_cats(test, train)

In [88]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 6 columns):
item_condition_id    1482535 non-null category
category_name        1476208 non-null category
brand_name           849853 non-null category
shipping             1482535 non-null category
x                    1482535 non-null float64
price                1482535 non-null float32
dtypes: category(4), float32(1), float64(1)
memory usage: 25.7 MB


In [89]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 7 columns):
item_condition_id    693359 non-null category
category_name        690271 non-null category
brand_name           397246 non-null category
shipping             693359 non-null category
x                    693359 non-null float64
price                693359 non-null int64
Id                   693359 non-null int64
dtypes: category(4), float64(1), int64(2)
memory usage: 20.1 MB


In [90]:
test.index

RangeIndex(start=0, stop=693359, step=1)

In [91]:
for v in contin_vars:
    train[v] = train[v].fillna(0).astype('float32')
    test[v] = test[v].fillna(0).astype('float32')

We're going to run on a sample.

In [92]:
val_idx = get_cv_idxs(n, val_pct=.1)
train_samp = train.iloc[val_idx]
samp_size = len(train_samp); samp_size

148253

In [93]:
train_samp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148253 entries, 777341 to 647202
Data columns (total 6 columns):
item_condition_id    148253 non-null category
category_name        147644 non-null category
brand_name           85145 non-null category
shipping             148253 non-null category
x                    148253 non-null float32
price                148253 non-null float32
dtypes: category(4), float32(2)
memory usage: 3.4 MB


To run on the full dataset, use this instead:

In [94]:
# samp_size = n
# train_samp = train

We can now process our data...

In [95]:
train_samp.head(2)

,item_condition_id,category_name,brand_name,shipping,x,price
777341,1,Handmade/Paper Goods/Stationery,NaN,1,0.705269,12.0
1463629,1,Women/Other/Other,Adidas,1,0.389329,76.0


In [96]:
train_samp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148253 entries, 777341 to 647202
Data columns (total 6 columns):
item_condition_id    148253 non-null category
category_name        147644 non-null category
brand_name           85145 non-null category
shipping             148253 non-null category
x                    148253 non-null float32
price                148253 non-null float32
dtypes: category(4), float32(2)
memory usage: 3.4 MB


In [97]:
# train_samp.reset_index(inplace=True, drop=True)

In [98]:
train_samp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148253 entries, 777341 to 647202
Data columns (total 6 columns):
item_condition_id    148253 non-null category
category_name        147644 non-null category
brand_name           85145 non-null category
shipping             148253 non-null category
x                    148253 non-null float32
price                148253 non-null float32
dtypes: category(4), float32(2)
memory usage: 3.4 MB


In [99]:
train_samp.head(2)

,item_condition_id,category_name,brand_name,shipping,x,price
777341,1,Handmade/Paper Goods/Stationery,NaN,1,0.705269,12.0
1463629,1,Women/Other/Other,Adidas,1,0.389329,76.0


In [100]:
df, y, nas, mapper = proc_df(train_samp,
                             y_fld = 'price',
                             do_scale=True)
yl = np.log(y)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


In [101]:
y

array([ 12.,  76.,  12.,   3.,  30., 225.,  16.,  12.,  15.,  17.,  28.,   9.,  19.,   4.,  14.,  71.,  10.,
        14.,  80.,  17., ...,  14.,  86.,   9.,  11.,  16.,  12.,  16.,  86.,  21.,  31.,  26.,  27.,   5.,
        12.,   4.,   5.,  14.,  16.,   7.,  22.], dtype=float32)

In [102]:
df.head()

,item_condition_id,category_name,brand_name,shipping,x
777341,1,331,0,2,0.712823
1463629,1,1194,103,2,-0.383355
350669,1,29,0,2,1.422954
310222,1,685,0,2,1.543253
759257,2,29,901,2,-0.096016


In [103]:
test.columns

Index(['item_condition_id', 'category_name', 'brand_name', 'shipping', 'x',
       'price', 'Id'],
      dtype='object')

In [104]:
# test.reset_index(inplace=True, drop=True)

In [105]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693359 entries, 0 to 693358
Data columns (total 7 columns):
item_condition_id    693359 non-null category
category_name        690271 non-null category
brand_name           397246 non-null category
shipping             693359 non-null category
x                    693359 non-null float32
price                693359 non-null int64
Id                   693359 non-null int64
dtypes: category(4), float32(1), int64(2)
memory usage: 17.4 MB


In [106]:
df_test, _, nas, mapper = proc_df(test, 'price',
                                  do_scale=True,
                                  skip_flds=['Id'],
                                  mapper=mapper,
                                  na_dict=nas)

In [107]:
df.head(2)

,item_condition_id,category_name,brand_name,shipping,x
777341,1,331,0,2,0.712823
1463629,1,1194,103,2,-0.383355


In time series data, cross-validation is not random. Instead, our holdout data is generally the most recent data, as it would be in real application. This issue is discussed in detail in [this post](http://www.fast.ai/2017/11/13/validation-sets/) on our web site.

One approach is to take the last 25% of rows (sorted by date) as our validation set.

In [108]:
# train_ratio = 0.75
# # train_ratio = 0.9
# train_size = int(samp_size * train_ratio); train_size
# val_idx = list(range(train_size, len(df)))

An even better option for picking a validation set is using the exact same length of time period as the test set uses - this is implemented here:

In [109]:
# val_idx = np.flatnonzero(
#     (df.index<=datetime.datetime(2014,9,17)) & (df.index>=datetime.datetime(2014,8,1)))

In [110]:
# val_idx = get_cv_idxs(samp_size)

## DL

We're ready to put together our models.

Root-mean-squared percent error is the metric Kaggle used for this competition.

In [111]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

def rmsle(y_pred, targ): 
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(targ))**2))


max_log_y = np.max(yl)
y_range = (0, max_log_y*1.2)

We can create a ModelData object directly from out data frame.

In [112]:
df_test.head()

,item_condition_id,category_name,brand_name,shipping,x
0,1,1183,0,2,1.431927
1,1,878,0,2,-1.454747
2,1,978,1010,2,-1.590224
3,2,1234,0,1,1.661097
4,3,840,0,2,-0.445694


In [113]:
cat_vars

['item_condition_id', 'category_name', 'brand_name', 'shipping']

In [114]:
val_idx

array([ 777341, 1463629,  350669,  310222,  759257,  288846, 1178450,  726296,  840510, 1473033, 1170796,
        506473, 1134656,  587885,   93928,  260369,   96202,  486334, 1437987, 1343222, ..., 1369016,
        118242,  401285, 1190911, 1238336,    5388,   95107,  416536,  192194, 1138567,  437357, 1075525,
       1149795,  378840, 1329473, 1472679, 1026985,  121050,  741085,  647202])

In [115]:
df.head()

,item_condition_id,category_name,brand_name,shipping,x
777341,1,331,0,2,0.712823
1463629,1,1194,103,2,-0.383355
350669,1,29,0,2,1.422954
310222,1,685,0,2,1.543253
759257,2,29,901,2,-0.096016


In [116]:
y1


NameError: name 'y1' is not defined

In [117]:
cat_vars

['item_condition_id', 'category_name', 'brand_name', 'shipping']

In [120]:
777341  in val_idx

True

In [121]:
df.shape

(148253, 5)

In [119]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, 
                                       yl.astype(np.float32),
                                       cat_flds=cat_vars, bs=128,
                                       test_df=df_test)

IndexError: index 777341 is out of bounds for axis 1 with size 148253

Some categorical variables have a lot more levels than others. Store, in particular, has over a thousand!

In [64]:
cat_sz = [(c, len(train_samp[c].cat.categories)+1) for c in cat_vars]

In [66]:
cat_sz

[('item_condition_id', 6),
 ('category_name', 1288),
 ('brand_name', 4810),
 ('shipping', 3)]

We use the *cardinality* of each variable (that is, its number of unique values) to decide how large to make its *embeddings*. Each level will be associated with a vector with length defined as below.

In [67]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [68]:
emb_szs

[(6, 3), (1288, 50), (4810, 50), (3, 2)]

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
m.summary()

In [ ]:
m.lr_find()

In [ ]:
m.sched.plot(100)

### Sample

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.fit(lr, 3, metrics=[rmsle])

In [ ]:
m.fit(lr, 5, metrics=[exp_rmspe], cycle_len=1)

In [ ]:
m.fit(lr, 2, metrics=[exp_rmspe], cycle_len=4)

### All

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.fit(lr, 1, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe], cycle_len=1)

### Test

In [ ]:
m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range)
lr = 1e-3

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe])

In [ ]:
m.fit(lr, 3, metrics=[exp_rmspe], cycle_len=1)

In [ ]:
m.save('val0')

In [ ]:
m.load('val0')

In [ ]:
x,y=m.predict_with_targs()

In [ ]:
exp_rmspe(x,y)

In [ ]:
pred_test=m.predict(True)

In [ ]:
pred_test = np.exp(pred_test)

In [ ]:
joined_test['Sales']=pred_test

In [ ]:
csv_fn=f'{PATH}tmp/sub.csv'

In [ ]:
joined_test[['Id','Sales']].to_csv(csv_fn, index=False)

In [ ]:
FileLink(csv_fn)

## RF

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
((val,trn), (y_val,y_trn)) = split_by_idx(val_idx, df.values, yl)

In [ ]:
m = RandomForestRegressor(n_estimators=40, max_features=0.99, min_samples_leaf=2,
                          n_jobs=-1, oob_score=True)
m.fit(trn, y_trn);

In [ ]:
preds = m.predict(val)
m.score(trn, y_trn), m.score(val, y_val), m.oob_score_, exp_rmspe(preds, y_val)